In [ ]:
# context is added to the prompt by retrieving data from an external source, in this case, Amazon Kendra, 
# a search engine service provided by AWS.
# for this exercise, customer service interaction notes have been generated using an LLM.
# these notes are for a fictitious ticketing company named 'ticketmom'. 
# the interactions have been indexed by Kendra and are ready for search.

# the code below retrieves search results from Kendra, injects them into a prompt, then asks a question 
# about the content.

In [ ]:
import boto3

from langchain_aws import ChatBedrock
from langchain_core.output_parsers import StrOutputParser

from langchain.prompts import PromptTemplate
from langchain_community.retrievers import AmazonKendraRetriever
from langchain.prompts.few_shot import FewShotPromptTemplate

In [ ]:
# looks up the id for the guardrail named 'sanitize'
bedrock_client = boto3.client('bedrock')
guardrails = bedrock_client.list_guardrails()
this_guardrail_id = None
for guardrail in guardrails['guardrails']:
    if guardrail['name'] == 'sanitizer':
        this_guardrail_id = guardrail['id']
        break
print("guardrail ID: " + this_guardrail_id)

# reference to the foundation model via amazon bedrock.
# https://docs.aws.amazon.com/bedrock/latest/userguide/inference-parameters.html
model = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",  # claude is awesome!    
    model_kwargs={
        "temperature": 0.95,
        "top_p": 0.95,
        "top_k": 250,
        "max_tokens": 20000
    },
    guardrails={                                         # guardrails filter prompts and responses
        "guardrailIdentifier": this_guardrail_id, 
        "guardrailVersion": "1"
    }
)

In [ ]:
# the following code is used to look up the id of the kendra index
# you probably don't need to do this in your own AWS account.

# create connection to AWS API using boto3
client = boto3.client('kendra')

# return a list of all indices
response = client.list_indices()

# iterate through the indices until you find the one named 'ticketmom'
# ..then assign the Id to index_id
index_id = None
for index in response['IndexConfigurationSummaryItems']:
    if index['Name'] == 'ticketmom':
        index_id = index['Id']

print(f"Amazon Kendra Index ID: {index_id}")

In [ ]:
# retrievers are langchain objects which retrieve from external data stores.
retriever = AmazonKendraRetriever(
    index_id=index_id,
    top_k = 5 # the number of search result documents to return
)

In [ ]:
# we are using a few-shot prompt template because it easily injects a list of records
# into a prompt. we are not doing few-shot prompting for this use case.

# this is the search term we will use when querying kendra
customer_service_issue = 'computer error'

# generates the prompt and injects the search results
prompt = FewShotPromptTemplate(
    examples=retriever.invoke(customer_service_issue),
    example_prompt=PromptTemplate(
        input_variables=["page_content"],
        template="{page_content}\n",
    ),
    suffix="""
act as an expert in customer service. 

assess the preceding customer interactions to identify how ticketmom could 
deliver better quality service.. with the goal of creating happier customers.

articulate your suggestion very succinctly, with three bullet points one sentence each.

"""
)

# let's have a look at the prompt:
print(prompt.format())

In [ ]:
chain = prompt | model | StrOutputParser()

In [ ]:
response = chain.invoke(input={})

print(response)